In [1]:
import random
import pandas as pd

# Parameters
ROAD_LEN = 1000
NUM_LANES = 3
CHANNELS = ['OCC1', 'OCC2', 'OCC3']
CFP_BASE = 7
MAX_BO = 10
VEHICLE_ID_COUNTER = 0
VEHICLE_BASE_COUNT = 10
WEATHER_CONDITIONS = ['Clear', 'Rain', 'Fog', 'Night']

# Config
config = {'BO': 6, 'SO': 0, 'MO': 2, 'CAP_reduction': True, 'SD_size': 60}

# Vehicle Class
class Vehicle:
    def __init__(self, vid):
        self.id = vid
        self.position = random.uniform(0, ROAD_LEN)
        self.lane = random.randint(0, NUM_LANES - 1)
        self.speed = random.uniform(10, 35)
        self.priority = random.random() < 0.2
        self.comm_ok = False

    def update_pos(self):
        self.position = (self.position + self.speed) % ROAD_LEN

def can_use_OCC(v, weather):
    if weather in ['Fog', 'Rain']:
        return False
    # Allow even priority vehicles if their speed is stable
    return v.speed <= 35 and abs(v.speed - 25) <= 10

def run_occ_dynavlc_sim(iterations=20):
    global VEHICLE_ID_COUNTER
    current_weather = 'Clear'
    for t in range(iterations):
        print(f"\n--- Beacon Interval {t+1} ---")

        # Random Weather
        if t % 5 == 0:
            current_weather = random.choice(WEATHER_CONDITIONS)

        # Random Vehicles
        traffic_level = random.choice(['light', 'normal', 'burst'])
        if traffic_level == 'light':
            vehicle_count = random.randint(5, 8)
        elif traffic_level == 'normal':
            vehicle_count = random.randint(10, 15)
        else:
            vehicle_count = random.randint(20, 30)

        vehicles = [Vehicle(VEHICLE_ID_COUNTER + i) for i in range(vehicle_count)]
        VEHICLE_ID_COUNTER += vehicle_count

        # Check OCC feasibility
        occ_vehicles = []
        for v in vehicles:
            v.comm_ok = can_use_OCC(v, current_weather)
            if v.comm_ok:
                occ_vehicles.append(v)

        R = len(occ_vehicles)
        C = len(CHANNELS)
        CFP_slots = CFP_BASE + (config['MO'] if config['CAP_reduction'] else 0)
        NCFP = C * CFP_slots
        avg_speed = sum(v.speed for v in vehicles) / len(vehicles)

        print(f"Weather: {current_weather}, Traffic: {traffic_level}")
        print(f"OCC-eligible Vehicles: {R}/{vehicle_count}, Avg Speed: {avg_speed:.1f} m/s")

        # === Dynamic Logic ===
        if R > NCFP:
            config['CAP_reduction'] = True
            config['MO'] += 1
            print("🔺 MO Increased due to high demand.")
        elif R < 0.5 * NCFP and config['MO'] > 0:
            config['MO'] -= 1
            print("🔻 MO Decreased due to underutilization.")
        elif R < 0.3 * NCFP:
            config['CAP_reduction'] = False
            print("🟡 CAP Reduction turned OFF due to low demand.")

        # Optional SO tuning based on speed
        if avg_speed > 25 and config['SO'] < config['BO']:
            config['SO'] += 1
        elif avg_speed < 15 and config['SO'] > 0:
            config['SO'] -= 1

        # Cap MO to BO
        if config['MO'] > config['BO']:
            config['MO'] = config['BO']

        print("Updated Config:", config)

        # Vehicle Print
        df = pd.DataFrame([{
            "ID": v.id, "Speed": round(v.speed, 1),
            "Priority": v.priority, "OCC_OK": v.comm_ok
        } for v in vehicles])
        print(df.to_string(index=False))

        for v in vehicles:
            v.update_pos()

run_occ_dynavlc_sim()


ModuleNotFoundError: No module named 'pandas'